## Recommender env: implementation

In [5]:
# HIDDEN
import gym

#### Implementation

- Now that 

#### Review: what does an env need?

In the previous module, we listed the following requirements for an RL environment:

- Observation space
- Action space
- Constructor
- `reset()` method
- `step()` method

We'll now create all of these for our recommender environment. Let's go!

#### Constructor

- Let's start designing our environment, which we'll call `SlateRecommender`.
- Starting with the constructor:

```python
class SlateRecommender(gym.Env):
    def __init__(self, env_config=None):
        ...
```

- The constructor will take in various parameters, and store them.
- We won't show all these lines, but they look something like this:

```python
self.num_candidates = env_config.get("num_candidates", 10)
```

#### Observation space

- The first step is to set the observation space. 
- Our observations are the _features of candidate items_.
- For simplicity, we'll assume only 1 feature.
- Here, our observations are continuous-valued, not discrete like Frozen Lake.
- If you'd like, pause for a moment and think about what the observation space should be.
  - You may wish to consult the gym library documentation [here](https://www.gymlibrary.ml/content/spaces/).

In [18]:
# TODO
# we could have a multiple choice in here
# I guess it's potentially fine to have tons of short notebooks in this module???
# might be a bit of a hassle for video recording?

#### Observation space

- We want the `Box` space:

In [8]:
# HIDDEN
num_candidates=10

In [9]:
observation_space = gym.spaces.Box(low=0, high=1, shape=(num_candidates,))

- Features are between 0 and 1, number of features is number of item candidates. 
- Here is an sample from the observation space:

In [10]:
observation_space.sample()

array([0.60701895, 0.5663801 , 0.28644493, 0.18979028, 0.6666804 ,
       0.0806279 , 0.06486645, 0.08660238, 0.86835325, 0.0801603 ],
      dtype=float32)

Looks good 👍

#### Action space

- In this environment, the action is the chosen item to recommend.
- If you'd like, pause for a moment and think about what the action space should be.
  - You may wish to consult the gym library documentation [here](https://www.gymlibrary.ml/content/spaces/).

#### Action space

Unlike the observation space, the action space _is_ discrete:

In [11]:
action_space = gym.spaces.Discrete(num_candidates)

Here are 15 randomly sampled actions:

In [17]:
[action_space.sample() for i in range(15)]

[8, 7, 9, 4, 4, 4, 8, 3, 6, 4, 9, 3, 5, 1, 5]

Looks good 👍

#### How

Before we implement `reset` and `step`, we'll create some helper methods:

- A method to sample the (simulated) item candidates
- 

#### Outline

- observation space, action space
- variable candidates
- turn on resampling (or just have this always on by defualt)
- increase slate size
- long term effect
- etc
- TODO IF LOW ON TIME: not great, but if needed,could even do the recommender as module 2, where we provide it. and then module 4 we start building it, and also experimenting with observation and reward shaping
- REWARD SHAPING 2 types:
  - help it learn
  - actually change what you care about
- examples here: ethical recommendations
- some sort of total reward for the whole episode, not interim rewards
- other stuff
- or right and the offline RL
